# Geotraces

In [ ]:
import pandas as pd 
from time import time

## Dos métodos diferentes para obtener la distancia Haversine entre dos puntos sobre la superficie terrestre

In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    print(c)
    # Radius of earth in kilometers is 6371
    km = 6371 * c
    return km

In [ ]:
haversine(-3.8196207, 40.4381311, -3.6503509, 40.5327412)

In [ ]:
from math import radians, degrees, sin, cos, asin, acos, sqrt
def great_circle(lon1, lat1, lon2, lat2):
    #print(lon1, lat1, lon2, lat2)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    a=sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2)
    if a>1:
        a=1
    a=acos(a)
    return 6371 *a
    #return 6371 * (
    #    acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
    #)

In [ ]:
great_circle(-3.8196207, 40.4381311, -3.6503509, 40.5327412)

Método para conectar puntos consecutivos obteniendo la velocidad media entre ambos

In [ ]:
def puntos_a_caminos(d):
    caminos=[]
    time_total=0
    dist_total=0
    speed=0
    for idx, row in d.iterrows():
        if idx>0:
            dist = great_circle(d.iloc[idx,5], d.iloc[idx,4], d.iloc[idx-1,5], d.iloc[idx-1,4])
            time = d.iloc[idx,0] - d.iloc[idx-1,0]
            if time>0:  # if time==0, speed=previous_speed
                speed = dist/time*3600  # conversion form seconds to hours
            time_total+=time
            dist_total+=dist
            accur = (d.iloc[idx, 6] + d.iloc[idx-1,6])/2
            offset = (d.iloc[idx, 7] + d.iloc[idx-1,7])/2
            caminos.append([dist, time, speed, accur, offset])
    d = pd.DataFrame(caminos, columns = ['Distance', 'Time (s)', 'Speed (km/h)', 'Accuracy', 'Offset']) 
    return d

# 1 user test

In [ ]:
start_time = time()
#-----------------------------------------------------------------
# read csv
filename = './users_Madrid/39.csv'
df = pd.read_csv(filename, sep = '\t')
#-----------------------------------------------------------------
total_time = time() - start_time
print(str(total_time) + " segundos")

In [ ]:
df.head(21)

In [ ]:
puntos_a_caminos(df)

### Printing with folium

In [ ]:
from datetime import datetime
import folium
from IPython.display import display

In [ ]:
#Funcion para dibujar las velocidades en el camino con diferentes colores
def speed_color(speed):
    if speed < 0:
        raise ValueError
    elif speed >= 0 and speed < 5:
        return 'red'
    elif speed >= 10 and speed < 60:
        return 'yellow'
    else:
        return 'green'

In [ ]:
userID = df['Device ID'][0]

#styles = ["Stamen Terrain", "Stamen Toner", "Mapbox Bright"]
points = df.head(47)
ways = puntos_a_caminos(df)
center = [40.4167278, -3.7033387] # Puerta del Sol (Madrid)
m = folium.Map(location=[center[0], center[1]], zoom_start=12)
    
for i in range(len(points)-1) : 
    dt_object = points.iloc[i, 1] 
    #Obtenemos dos ubicaciones para poder ir dibujando la linea que los une (poligono)
    p1 = [points.iloc[i, 4], points.iloc[i, 5]]
    p2 = [points.iloc[i+1, 4], points.iloc[i+1, 5]]
    speed = round(ways['Speed (km/h)'][i], 2) #Redondeamos los decimales de la velocidad
    folium.PolyLine(locations=[p1, p2], color=speed_color(speed), tooltip=str(speed) +' km/h').add_to(m)
    if i==0:
        folium.Marker(location=p1,popup= 'Punto de inicio: ' + str(dt_object), icon=folium.Icon(color='green')
                 ).add_to(m)
    elif i == len(points)-2:
        folium.Marker(location=p2,popup= 'Punto final: ' + str(dt_object), icon=folium.Icon(color='red')
                 ).add_to(m)
    else:
        folium.Circle(radius=20,location=p1,popup=dt_object,color='orange',
                 ).add_to(m)
        
display(m)

# N users

In [ ]:
import glob

In [ ]:
start_time = time()
#-----------------------------------------------------------------

path = './users_Baleares_tags' # use your path
all_files = glob.glob(path + "/*.csv")

for csv_user in all_files:
    df_aux = pd.read_csv(csv_user, sep = '\t')
    user = csv_user.split('/')[-1]
    
    speed = puntos_a_caminos(df_aux)['Speed (km/h)']
    df_aux['Speed (km/h)'] = 0
    for i, row in df_aux.iterrows():
        if i>0:
             df_aux.iloc[i,-1] = speed[i-1]
    df_aux.to_csv("./users_Baleares_tags_speed/"+user, sep='\t', index=False)
                
#-----------------------------------------------------------------
total_time = time() - start_time
print(str(total_time) + " segundos")

### Plot clustering 1 user

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
%matplotlib inline
# define the number of kilometers in one radian
kms_per_radian = 6371.0088

In [ ]:
start_time = time()
#-----------------------------------------------------------------
# read csv
df = pd.read_csv('./users_Madrid_speed/9684.csv', sep = '\t')
#-----------------------------------------------------------------
total_time = time() - start_time
print(str(total_time) + " segundos")

In [ ]:
len(df)

In [ ]:
df_cluster = df[['Latitude','Longitude']]

In [ ]:
kmeans = KMeans(n_clusters=2).fit(df_cluster)
centroids = kmeans.cluster_centers_
print(centroids)

plt.scatter(df_cluster['Longitude'], df_cluster['Latitude'],c=kmeans.labels_.astype(float), s=1, alpha=0.8)
plt.scatter(centroids[:, 1], centroids[:, 0], c='red', s=50)

In [ ]:
# scatterplot it to get a sense of what it looks like
df_cluster = df_cluster.sort_values(by=['Latitude', 'Longitude'])
ax = df_cluster.plot(kind='scatter', x='Longitude', y='Latitude', alpha=0.5, linewidth=0)

# Detección de estados de Markov

#### El obj

In [ ]:
from time import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import math

In [ ]:
states_id = ['En reposo', 'Andando', 'Bus', 'Tren', 'Coche']
def get_Markov_states(df):
    if len(df)<2:
        return []
    states = [[str(df.iloc[0, 1].hour) + ':' + str(df.iloc[0, 1].minute) , 'En reposo', 
               get_mins(df.iloc[1, 0] - df.iloc[0, 0])]]
    all_states = [[str(df.iloc[0, 1].hour) + ':' + str(df.iloc[0, 1].minute) , 'En reposo', 
               get_mins(df.iloc[1, 0] - df.iloc[0, 0])]]
    speeds = []
    sum_time = 0
    
    # Reseteamos los indices
    df = df.reset_index(drop=True)
    for i, row in df.iterrows():
        if i+1 == len(df): break
        speed = row['Speed (km/h)']
        time = str(df.iloc[i, 1].hour) + ':' + str(df.iloc[i, 1].minute)
        time_length = get_mins(df.iloc[i+1, 0] - df.iloc[i, 0])
        
        # STOPPED
        if speed < 2.78: 
            if all_states[-1][1] != 'En reposo':
                states.append([time, 'En reposo', time_length])
            else: 
                states[-1][2] += time_length
            all_states.append([time, 'En reposo', time_length])
        # WALK
        elif speed <= 8:
            if all_states[-1][1] != 'Andando':
                states.append([time, 'Andando', time_length])
            else: 
                states[-1][2] += time_length
            all_states.append([time, 'Andando', time_length])
        # TRAIN
        elif ((row['Train'] == True and speed > 8) 
            or (all_states[-1][1] == 'Tren' and speed > 8)):
            if all_states[-1][1] != 'Tren':
                states.append([time, 'Tren', time_length])
            else: 
                states[-1][2] += time_length
            all_states.append([time, 'Tren', time_length])
        # BUS
        elif ((row['Bus'] == True and speed > 8) 
            or (all_states[-1][1] == 'Bus' and speed > 8)):
            if all_states[-1][1] != 'Bus':
                states.append([time, 'Bus', time_length])
            else: 
                states[-1][2] += time_length
            all_states.append([time, 'Bus', time_length])
        # CAR
        elif speed > 15 or row['Fuel Station'] == True or (all_states[-1][1] == 'Coche' and speed > 15):
            if all_states[-1][1] != 'Coche':
                states.append([time, 'Coche', time_length])
            else: 
                states[-1][2] += time_length
            all_states.append([time, 'Coche', time_length])
    return states

def update_transition_matrix(matrix, transitions):
    if len(transitions)>1:
        for i in range(len(transitions)-1):
            matrix.loc[transitions[i][1], transitions[i+1][1]] += 1
            
def calculate_prob(matrix):
    for i in matrix.index:
        total = matrix.loc[i, :].sum()
        for j in matrix.columns:
            # Usamos estimador bayesiano
            if total != 0:
                matrix.loc[i, j] = (matrix.loc[i, j] + 1) / (total + len(states_id))
            else:
                matrix.loc[i, j] = 0
            
def get_mins(tstmp):
    mins = math.floor(tstmp/60)
    return mins

In [ ]:
start_time = time()
#-----------------------------------------------------------------

print_figures = False

matrix_by_user = {}

# Matriz de transicion del usuario vacia
total_transition_matrix = pd.DataFrame(0, index=states_id, columns=states_id)

path = './users_Barcelona_tags_speed' # use your path
all_files = glob.glob(path + "/*.csv")

for csv_user in all_files:
    #if csv_user == './users_Madrid_tags_speed/106180.csv':
    df = pd.read_csv(csv_user, sep = '\t')
    user = csv_user.split('/')[-1]
    user = user.split('.')[0]
    print('User ID: '+str(user))

    # Matriz de transicion del usuario vacia
    transition_matrix = pd.DataFrame(0, index=states_id, columns=states_id)

    # Se crea una columna con la fecha legible
    df['Date Time'] = pd.to_datetime(df['Date Time'], format="%Y-%m-%d %H:%M:%S")

    for i in range(2, 30):
        #Ploteamos una grafica para cada dia del mes 7 (julio)
        df_plot = df[(df['Date Time']<datetime(2019, 7, i, 0, 0)) & (df['Date Time']>datetime(2019, 7, i-1, 0, 0))]

        if len(df_plot) < 5:
            continue

        # Obtenemos las transiciones del usuario
        states = get_Markov_states(df_plot)
        #print(states)

        # Actualizamos la matriz de transicion
        update_transition_matrix(transition_matrix, states)
        update_transition_matrix(total_transition_matrix, states)

        if print_figures == True:
            #Creamos el grafico
            x = df_plot['Date Time'].tolist()
            y = df_plot['Speed (km/h)'].tolist()
            plt.figure(figsize=(20,5))
            plt.step(x, y, where = 'mid', color = 'blue', linewidth = 2) 
            plt.xticks(rotation=20)
            plt.title(datetime(2019, 7, i-1).date())
            plt.xlabel('Hour')
            plt.ylabel('km/h')
            plt.ylim(0, 100)
            plt.grid()

            #BUS
            line_bus=0
            #Comprobamos para qué puntos hay True en tag=Bus
            df_bus = df_plot[df_plot['Bus']==True]
            if df_bus.empty == False:
                #Generamos linea ROJA discontinua para puntos que tienen tag_bus = True
                for index, item in enumerate(x):
                    if item in df_bus['Date Time'].tolist():
                        line_bus = plt.axvline(x[index], color='g', ls="dotted", linewidth = 3)
                line_bus.set_label('Bus stop')

            #TRAIN
            line_train=0
            #Comprobamos para qué puntos hay True en tag=Train
            df_train = df_plot[df_plot['Train']==True]
            if df_train.empty == False:
                #Generamos linea ROJA discontinua para puntos que tienen tag_bus = True
                for index, item in enumerate(x):
                    if item in df_train['Date Time'].tolist():
                        line_train = plt.axvline(x[index], color='m', ls="dotted", linewidth = 3)
                line_train.set_label('Train station')

            #Fuel Station
            #Comprobamos para qué puntos hay True en tag=Fuel Station
            df_fuelst = df_plot[df_plot['Fuel Station']==True]
            if df_fuelst.empty == False:
                #Generamos linea ROJA discontinua para puntos que tienen tag_bus = True
                for index, item in enumerate(x):
                    if item in df_fuelst['Date Time'].tolist():
                        line_fuel = plt.axvline(x[index], color='r', ls="dotted", linewidth = 3)
                line_fuel.set_label('Fuel Station')

            plt.legend(loc='best', borderaxespad=0)

            plt.show()

    calculate_prob(transition_matrix)
    matrix_by_user[user] = transition_matrix
    #print(transition_matrix)
    #print('\n')
    
calculate_prob(total_transition_matrix)
print('\nMatriz de transición del conjunto total de usuarios:')
print(total_transition_matrix)

#-----------------------------------------------------------------
total_time = time() - start_time
print(str(total_time) + " segundos")

## Cálculo de la distribución estacionaria

### Método 1: Cálculo de la potencia n de la matriz de transición para conocer el estado estacionario

In [ ]:
def get_stable_matrix(transition_matrix):
    # Pasamos la matriz a un array en numpy
    matriz = transition_matrix.to_numpy()
    n=500
    pn=np.linalg.matrix_power(matriz, n)
    return pn

In [ ]:
# Obtenemos los vectores estacionarios de todos los usuarios
cluster_data = []
for m in matrix_by_user.values():
    bandera = True
    M = get_stable_matrix(m)
    for i in M:
        if round(i.sum()) != 1:
            bandera = False
            continue
    if bandera:
        cluster_data.append(M[0])
cluster_data

### Método 2: Resolución de la ecuación de distribución estacionaria

In [ ]:
def get_stable_state(transition_matrix):
    # Resolviendo por matrices A = (AT-I) y el vector de ceros terminado en 1
    matriz = transition_matrix.to_numpy()
    k=len(matriz)
    A=matriz.transpose()
    A=A-np.identity(k, dtype=int)
    # la última fila se sustituye por la suma de probabilidades
    A[-1,:]=np.ones(k,dtype=int)
    B=np.zeros(k,dtype=int)
    B[-1]=1  # el último
    Pncalc=np.linalg.solve(A,B)
    #print('largo plazo')
    #print(Pncalc)
    return Pncalc

## Obtenemos los vectores estacionarios de cada usuario

In [ ]:
# Obtenemos los vectores estacionarios de todos los usuarios
cluster_data = []
for m in matrix_by_user.values():
    bandera = True
    v = get_stable_state(m)
    for i in v:
        if i == 0:
            bandera = False
            continue
    if bandera:
        cluster_data.append(v.tolist())
cluster_data

## Clustering de vectores estacionarios

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
Sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k, random_state=0)
    km = km.fit(cluster_data)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('Número de clusters')
plt.ylabel('SSE')
plt.title('Elbow Method')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(cluster_data)
centroids = kmeans.cluster_centers_
#print(centroids)
#print(kmeans.labels_)

state_X = 0
state_Y = 1

X = []
Y = []
for user in cluster_data:
    X.append(user[state_X])
    Y.append(user[state_Y])

plt.scatter(X, Y, s=10, alpha=0.8, c=kmeans.labels_.astype(float))
plt.scatter(centroids[:, state_X], centroids[:, state_Y], c =range(len(centroids)), s=100)
plt.xlabel('Probabilidad del estado: '+states_id[state_X])
plt.ylabel('Probabilidad del estado: '+states_id[state_Y])
plt.show()

In [ ]:
list(zip(cluster_data, kmeans.labels_))
#[0 1 1 3 1 3 1 1 3 3 2 1 3 1 3 1 1 1 2 2 2 1 3 1 1 3 2 0 3 1 3 2 3 2 0 1 0
# 1 3 1 3 3 1 1]

## Clustering sobre mapa 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
%matplotlib inline
# define the number of kilometers in one radian
kms_per_radian = 6371.0088

In [ ]:
start_time = time()
#-----------------------------------------------------------------
# read csv
df_raw = pd.read_csv('./users_Madrid_tags_speed/17057.csv', sep = '\t')
# Se añade la columna Date Time para la fecha en formato legible
df['Date Time'] = pd.to_datetime(df['Date Time'], format="%Y-%m-%d %H:%M:%S")
#-----------------------------------------------------------------
total_time = time() - start_time
print(str(total_time) + " segundos")

In [ ]:
mapa = plt.imread('./map.png')

In [ ]:
BBox = (-3.9087, -3.4555, 40.3311, 40.5313)

In [ ]:
fig, ax = plt.subplots(figsize = (15,8))

ax.scatter(df_raw.Longitude, df_raw.Latitude, zorder=1, alpha= 0.8, c='b', s=10)
ax.set_title('Plotting Madrid Data')
ax.set_xlim(BBox[0], BBox[1])
ax.set_ylim(BBox[2], BBox[3])

ax.imshow(mapa, zorder=0, extent = BBox, aspect= 'equal')

In [ ]:
df_cluster = df_raw[['Latitude','Longitude']]
kmeans = KMeans(n_clusters=3).fit(df_cluster)
centroids = kmeans.cluster_centers_
print(centroids)

plt.scatter(df_raw.Longitude, df_raw.Latitude, c=kmeans.labels_.astype(float), s=1, alpha=0.8)
plt.scatter(centroids[:, 1], centroids[:, 0], c='red', s=50)